# 🎬 YouTube Content Integrity Analyzer

This notebook uses PraisonAI to analyze YouTube video captions for harmful content (hate speech, misinformation, violence, fraud) and evaluates educational value, relevance, integrity, and clarity. The agent provides unbiased, category-based content quality assessments using only the transcript text.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/Qwen2.5_InstructionAgent.ipynb)

# Dependencies

In [ ]:
!pip install praisonaiagents openai youtube-transcript-api --quiet

# Set up API KEY

In [ ]:
from praisonaiagents import Agent
import os
import json

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "Enter your api key"  # <-- Replace with your actual OpenAI API key

# Get YouTube Transcript

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_youtube_transcript(video_id, languages=['en']):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
        text = " ".join([entry['text'] for entry in transcript])
        return text
    except Exception as e:
        return None

# Prompts (role, goal, instructions)

In [ ]:
ROLE = """
Act as a content integrity and quality analyst trained to evaluate YouTube video captions. You are an expert in content policy enforcement, linguistic analysis, and media integrity.
You specialize in detecting harmful content (e.g., hate speech, misinformation, fraud) and evaluating the clarity, educational value, and relevance of language in video captions, using only textual evidence and without assumptions about tone or intent.
"""

GOAL = """
Your task is to assess whether specific content categories are present based solely on the language in the transcript. Your judgments should be precise, unbiased, and strictly based on linguistic clues — do not rely on assumptions about video intent or tone.
"""

INSTRUCTIONS = """
You will receive a list of categories and a YouTube transcript. Analyze the transcript to determine the presence of the specified categories.
Only use these categories: ["hatred", "misinformation", "violence", "fraud", "educational", "relevance", "integrity", "clarity"].
Discard any other category. If none remain, return an error.
For each valid category, assign a score from 0 to 10 and provide a reason.
Return a JSON object with:
  - categories: list of {name, score, connotation, reason}
  - overall: {score, reason}
  - error: ""
  - content_summary: brief summary of the transcript
Start your response with { and end with }.
"""

# Main Agent Function

In [ ]:
def analyze_youtube_content(video_url, categories, custom_prompts=None):
    # Extract video ID from URL
    import re
    match = re.search(r"v=([A-Za-z0-9_-]+)", video_url)
    if not match:
        return {"error": "Invalid YouTube URL"}
    video_id = match.group(1)

    transcript = get_youtube_transcript(video_id)
    if not transcript:
        return {"error": "Could not fetch transcript for this video."}

    # Prepare the prompt for the agent
    prompt = f"Categories: {categories}\n\nTranscript:\n{transcript}\n"
    if custom_prompts:
        for cp in custom_prompts:
            prompt += f"\nAdditional user prompt: {cp}"

    agent = Agent(
        role=ROLE,
        goal=GOAL,
        instructions=INSTRUCTIONS,
    )
    response = agent.start(prompt)
    return response

# Example Usage (with a valid video URL)

In [ ]:
# Example usage with a valid video URL (TED Talk with English transcript)
video_url = "https://www.youtube.com/watch?v=Ks-_Mh1QhMc"  # TED: Your body language may shape who you are
categories = ["hatred", "educational", "clarity"]
result = analyze_youtube_content(video_url, categories)
print(result)

╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Agent                                                                                                │
│  Role:                                                                                                          │
│  Act as a content integrity and quality analyst trained to evaluate YouTube video captions. You are an expert   │
│  in content policy enforcement, linguistic analysis, and media integrity.                                       │
│  You specialize in detecting harmful content (e.g., hate speech, misinformation, fraud) and evaluating the      │
│  clarity, educational value, and relevance of language in video captions, using only textual evidence and       │
│  without assumptions about tone or intent.                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 6.0s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ Categories: ['hatred', 'educational', 'clarity']                                                                │
│                                                                                                                 │
│ Transcript: Translator: Joseph Geni Reviewer: Morton Bast So I want to start by offering you a free no-tech     │
│ life hack, and all it requires of you is this: that you change your posture for two minutes. But before I give  │
│ it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So │
│ how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe         │
│ wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out. (Laughter) I see you. │
│ So I want you to pay attention to what you're doing right now. We're going to come back to that in a few        │
│ minutes, and I'm hoping that if you learn to tweak this a little bit, it could significantly change the way     │
│ your life unfolds. So, we're really fascinated with body language, and we're particularly interested in other   │
│ people's body language. You know, we're interested in, like, you know — (Laughter) — an awkward interaction, or │
│ a smile, or a contemptuous glance, or maybe a very awkward wink, or maybe even something like a handshake.      │
│ Narrator: Here they are arriving at Number 10. This lucky policeman gets to shake hands with the President of   │
│ the United States. Here comes the Prime Minister -- No. (Laughter) (Applause) (Laughter) (Applause) Amy Cuddy:  │
│ So a handshake, or the lack of a handshake, can have us talking for weeks and weeks and weeks. Even the BBC and │
│ The New York Times. So obviously when we think about nonverbal behavior, or body language -- but we call it     │
│ nonverbals as social scientists -- it's language, so we think about communication. When we think about          │
│ communication, we think about interactions. So what is your body language communicating to me? What's mine      │
│ communicating to you? And there's a lot of reason to believe that this is a valid way to look at this. So       │
│ social scientists have spent a lot of time looking at the effects of our body language, or other people's body  │
│ language, on judgments. And we make sweeping judgments and inferences from body language. And those judgments   │
│ can predict really meaningful life outcomes like who we hire or promote, who we ask out on a date. For example, │
│ Nalini Ambady, a researcher at Tufts University, shows that when people watch 30-second soundless clips of real │
│ physician-patient interactions, their judgments of the physician's niceness predict whether or not that         │
│ physician will be sued. So it doesn't have to do so much with whether or not that physician was incompetent,    │
│ but do we like that person and how they interacted? Even more dramatic, Alex Todorov at Princeton has shown us  │
│ that judgments of political candidates' faces in just one second predict 70 percent of U.S. Senate and          │
│ gubernatorial race outcomes, and even, let's go digital, emoticons used well in online negotiations can lead    │
│ you to claim more value from that negotiation. If you use them poorly, bad idea. Right? So when we think of     │
│ nonverbals, we think of how we judge others, how they judge us and what the outcomes are. We tend to forget,    │
│ though, the other audience that's influenced by our nonverbals, and that's ourselves. We are also influenced by │
│ our nonverbals, our thoughts and our feelings and our physiology. So what nonverbals am I talking about? I'm a  │
│ social psychologist. I study prejudice, and I teach at a competitive business school, so it was inevitable that │
│ I would become interested in power dynamics. I became 

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ { "categories": [ { "name": "educational", "score": 9, "connotation": "positive", "reason": "The transcript     │
│ provides a detailed explanation of body language and its effects on personal and professional outcomes. It      │
│ includes scientific studies and practical advice, making it highly educational." }, { "name": "clarity",        │
│ "score": 8, "connotation": "positive", "reason": "The transcript is clear in its explanation of concepts        │
│ related to body language and power dynamics. It uses examples and studies to illustrate points, although the    │
│ length and detail might be overwhelming for some." } ], "overall": { "score": 8.5, "reason": "The transcript is │
│ highly educational and clear, providing valuable insights into body language and its impact on personal         │
│ empowerment. It effectively communicates scientific findings and practical applications." }, "error": "",       │
│ "content_summary": "The transcript discusses the impact of body language on personal empowerment and            │
│ professional success. It explains scientific studies on nonverbal communication, power dynamics, and the        │
│ concept of 'faking it till you become it.' Practical advice is given on how to use body language to improve     │
│ self-confidence and outcomes in evaluative situations." }                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{
  "categories": [
    {
      "name": "educational",
      "score": 9,
      "connotation": "positive",
      "reason": "The transcript provides a detailed explanation of body language and its effects on personal and professional outcomes. It includes scientific studies and practical advice, making it highly educational."
    },
    {
      "name": "clarity",
      "score": 8,
      "connotation": "positive",
      "reason": "The transcript is clear in its explanation of concepts related to body language and power dynamics. It uses examples and studies to illustrate points, although the length and detail might be overwhelming for some."
    }
  ],
  "overall": {
    "score": 8.5,
    "reason": "The transcript is highly educational and clear, providing valuable insights into body language and its impact on personal empowerment. It effectively communicates scientific findings and practical applications."
  },
  "error": "",
  "content_summary": "The transcript discusses the impact of b